Given an m x n board of characters and a list of strings words, return all words on the board.

Each word must be constructed from letters of sequentially adjacent cells, where adjacent cells are horizontally or vertically neighboring. The same letter cell may not be used more than once in a word.

 

Example 1:


Input: board = [["o","a","a","n"],["e","t","a","e"],["i","h","k","r"],["i","f","l","v"]], words = ["oath","pea","eat","rain"]
Output: ["eat","oath"]
Example 2:


Input: board = [["a","b"],["c","d"]], words = ["abcb"]
Output: []
 

Constraints:

m == board.length
n == board[i].length
1 <= m, n <= 12
board[i][j] is a lowercase English letter.
1 <= words.length <= 3 * 104
1 <= words[i].length <= 10
words[i] consists of lowercase English letters.
All the strings of words are unique.

In [ ]:
# brute force wourd be to call the backtracking solution for each dictionary word.
# 


The **Trie in Word Search II** is mainly for **prefix pruning**:

* Without Trie → for each DFS path, you’d have to check `if prefix in any word?` → costly (`O(N * L)` each time).
* With Trie → you know **immediately** if a path is invalid (since you stop DFS when `char not in node.children`).

So:

* ✅ **Avoids re-searching** all words for every path.
* ✅ Ensures you only explore board paths that can actually lead to a word.
* ✅ Makes the solution scale to large inputs (hundreds of words, big boards).



In [ ]:
from typing import List

class Node:
    def __init__(self):
        self.children = {}
        self.word = None   # Marks the end of a word


class Tries:
    def __init__(self):
        self.root = Node()

    def insert(self, word: str):
        node = self.root
        for ch in word:
            if ch not in node.children:
                node.children[ch] = Node()
            node = node.children[ch]
        node.word = word  # Mark complete word at end node


class Solution:
    def findWords(self, board: List[List[str]], words: List[str]) -> List[str]:
        # Build Trie
        trie = Tries()
        for word in words:
            trie.insert(word)

        rows, cols = len(board), len(board[0])
        result = []

        def dfs(r, c, node):
            char = board[r][c]
            if char not in node.children:
                return

            nxt_node = node.children[char]

            # ✅ Found a complete word
            if nxt_node.word:
                result.append(nxt_node.word)
                nxt_node.word = None  # avoid duplicates

            # Mark visited
            board[r][c] = "#"

            # Explore neighbors
            for dr, dc in [(1,0), (-1,0), (0,1), (0,-1)]:
                nr, nc = r + dr, c + dc
                if 0 <= nr < rows and 0 <= nc < cols and board[nr][nc] != "#":
                    dfs(nr, nc, nxt_node)

            # Backtrack
            board[r][c] = char

            # ✅ Optimization: prune dead branch # NOTE
            if not nxt_node.children:
                node.children.pop(char)

        # Start DFS from every cell
        for r in range(rows):
            for c in range(cols):
                dfs(r, c, trie.root)

        return result



* **Build Trie:** `O(N × L)`

  * `N` = number of words
  * `L` = average word length
* **DFS search:** each board cell starts DFS, but search prunes **as soon as no prefix exists**.

  * Worst case: visit each cell and explore up to 4 directions at most `L` deep.
  * `O(M × 4^L)` → but much less in practice because Trie kills dead branches quickly.
  * Effectively bounded by number of valid prefixes in Trie.

👉 **Total Time:** `O(N × L + M × 4^L)` (but practically far smaller than this).
👉 **Space:**

* Trie = `O(N × L)`
* Recursion stack = `O(L)`
* Answer storage = `O(K × L)` (`K` = number of found words).

